In [47]:
import heapq
from collections import deque
class Solution:
    '''
        官方思路：https://leetcode.cn/problems/sliding-window-maximum/solutions/543426/hua-dong-chuang-kou-zui-da-zhi-by-leetco-ki6m/
        解题思路：大顶堆控制最大值
            1、 用大顶堆解决此题的唯一难点便是，大顶堆的堆顶元素何时删除，思路就是记录下标，在后续对下标已经离开窗口的堆顶元素进行出堆。
            但依单纯顺序出离开窗口的堆顶元素，有可能堆中仍然存在已经离开窗口的元素，但由于它总不是堆顶元素，故它必然不是每次窗口中的最大值，故不会影响堆顶，可以不需要对它出堆
            2、python使用堆需要注意默认是小顶堆，故需要加负号
    '''
    def maxSlidingWindow(self, nums: list[int], k: int) -> list[int]:
        n = len(nums)
        q = [(-nums[i],i) for i in range(k)]
        # python 默认是小顶堆，需要给数值加负号，转变为大顶堆
        heapq.heapify(q)

        # 取第一个窗口中，第一个最大值
        res = [-q[0][0]]

        # 从k往后遍历
        for i in range(k,n):
            heapq.heappush(q,(-nums[i],i))

            # 对于已经离开窗口的元组出队
            while q[0][1] <= i - k:
                heapq.heappop(q)
            
            # 取当前窗口的最大值
            res.append(-q[0][0])
        
        return res
    '''
        同官方思路
        解题思路：单调双端队列
            1、对于一个窗口中，假设下标 i和j (i < j)，有 nums[i] <= nums[j]，那么窗口中 nums[i] 失去参考价值。
            原因是，在当前窗口中，最大的元素必然不是nums[i]，有可能是nums[j]（等于情况也是一般情况不影响）；而后续移动窗口中，
            由于nums[i]在nums[j]的左边，则它必然会先于nums[j]离开窗口，故此时只需要保留nums[j]参考。
            2、使用单调双端队列，存储下标，单调递增，每次添加下标进入队列前，对队列重复做1操作，
            这样队头下标所对应的nums元素必然是当前窗口的最大值。
            3、与方法1同样需要注意，关于队头下标所对应的nums元素仍有可能已经离开了窗口，故需要做同样的操作，对其出队删除

    '''
    def maxSlidingWindow1(self, nums: list[int], k: int) -> list[int]:
        n = len(nums)
        
        # 创建单调双端队列，单调递增存储下标
        q = deque()

        # 对于第一个窗口处理
        for i in range(k):
            # 当队列不为空且队尾元素下标对应于nums的元素a 小于或等于 当前nums元素b时，
            # 说明当前窗口，元素a在元素b的左边 且 a <= b，故a失去参考价值，可以删除
            while q and nums[i] >= nums[q[-1]]:
                # deque.pop() 是对双端队列的队尾出队
                q.pop()
            q.append(i)
        
        # 由于上面处理，可以得出队头下标对应元素必然是当前窗口最大值
        res = [nums[q[0]]]

        # 开始移动窗口
        for i in range(k,n):
            # 与上操作相同
            while q and nums[i] >= nums[q[-1]]:
                q.pop()
            q.append(i)
            
            # 与方法1相同操作，删除以及离开窗口中的队头元素
            while q[0] <= i - k:
                # deque.popleft() 是对双端队列的队头出队
                q.popleft()
            
            res.append(nums[q[0]])

        return res

    '''
        方案2优化
        解题思路：
            1、利用方案2的思路，直接操作nums元素，而不利用其下标。
            2、在移动窗口中，利用zip函数，直接将窗口中的 首元素 与 尾元素 组成元组，做方案2中类似操作：
                对于队列中队尾元素小于等于当前窗口的尾元素，失去价值移除。
                对于队头元素（仍是当前窗口最大值）加入解集后，如果等于当前窗口的首元素，则说明移动窗口后，它可以直接删除
    '''
    def maxSlidingWindow2(self, nums: list[int], k: int) -> list[int]:
        # 仍是双端队列，但是直接存储nums中的元素
        q = deque()
        res = []

        # 对第一个窗口处理，但窗口中的尾元素先不做处理，留到下面开始移动窗口时再处理
        for i in range(k-1):
            # 队列存在且队尾元素小于当前nums元素，失去参考价值则删除
            while q and q[-1]<nums[i]:
                q.pop()
            q.append(nums[i])
        
        # zip(a,b) 可以取两个列表a和b中的元素，组成一个个元组，长度取最短
        # 移动窗口，pop为窗口头元素，push为窗口尾元素
        for pop, push in zip(nums, nums[k-1:]):
            # 同上操作
            while q and q[-1]<push:
                q.pop()
            q.append(push)
            res.append(q[0])

            # 队头元素为此窗口的元素值时可以出队
            if pop == q[0]:
                q.popleft()
        return res


In [45]:
def main():
    nums = [1,3,-1,-3,5,3,6,7]
    k = 3
    s = Solution()

    answer1 = s.maxSlidingWindow(nums,k)
    answer2 = s.maxSlidingWindow1(nums,k)
    answer3 = s.maxSlidingWindow2(nums,k)
    print(answer1)
    print(answer2)
    print(answer3)

In [48]:
if __name__ == "__main__":
    main()

[3, 3, 5, 5, 6, 7]
[3, 3, 5, 5, 6, 7]
[3, 3, 5, 5, 6, 7]
